## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [10]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model = keras.applications.xception.Xception(include_top=False, weights='imagenet', 
                                             input_tensor=input_tensor,
                                             pooling=None, classes=1000)
'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

## 添加層數

In [11]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x=Dropout(p=0.1)(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 136


C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


## 鎖定特定幾層不要更新權重

In [12]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [13]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [14]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

W1224 00:07:55.897131 16892 deprecation_wrapper.py:119] From C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1224 00:07:56.036758 16892 deprecation.py:323] From C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
50000/50000 [==============================] - 45s 892us/step - loss: 1.3868 - acc: 0.5170
Epoch 2/100
50000/50000 [==============================] - 37s 731us/step - loss: 1.0095 - acc: 0.6562
Epoch 3/100
50000/50000 [==============================] - 37s 731us/step - loss: 0.9023 - acc: 0.6896
Epoch 4/100
50000/50000 [==============================] - 36s 724us/step - loss: 0.8214 - acc: 0.7147
Epoch 5/100
50000/50000 [==============================] - 36s 730us/step - loss: 0.7435 - acc: 0.7452
Epoch 6/100
50000/50000 [==============================] - 35s 703us/step - loss: 0.6792 - acc: 0.7671
Epoch 7/100
50000/50000 [==============================] - 35s 703us/step - loss: 0.6098 - acc: 0.7896
Epoch 8/100
50000/50000 [==============================] - 35s 703us/step - loss: 0.5446 - acc: 0.8120
Epoch 9/100
50000/50000 [==============================] - 35s 702us/step - loss: 0.4961 - acc: 0.8303
Epoch 10/100
50000/50000 [==============================] - 35s 703us/ste

50000/50000 [==============================] - 37s 744us/step - loss: 0.0704 - acc: 0.9773
Epoch 80/100
50000/50000 [==============================] - 37s 741us/step - loss: 0.0729 - acc: 0.9759
Epoch 81/100
50000/50000 [==============================] - 37s 743us/step - loss: 0.0671 - acc: 0.9779
Epoch 82/100
50000/50000 [==============================] - 37s 750us/step - loss: 0.0729 - acc: 0.9772
Epoch 83/100
50000/50000 [==============================] - 37s 742us/step - loss: 0.0676 - acc: 0.9777
Epoch 84/100
50000/50000 [==============================] - 37s 739us/step - loss: 0.0662 - acc: 0.9788
Epoch 85/100
50000/50000 [==============================] - 37s 737us/step - loss: 0.0680 - acc: 0.9779
Epoch 86/100
50000/50000 [==============================] - 37s 738us/step - loss: 0.0651 - acc: 0.9788
Epoch 87/100
50000/50000 [==============================] - 37s 739us/step - loss: 0.0668 - acc: 0.9785
Epoch 88/100
50000/50000 [==============================] - 37s 737us/step - 